In [8]:
import pandas as pd
import numpy as np
import glob
import os
import torch.nn.functional as F
import torch

In [2]:
dir = './data/comments'
datasets_dir = "./data/datasets"

files = sorted(glob.glob(f'{dir}/*.pickle'))

print(files)

['./data/comments/20200201_20200315_all_comments_with_labels.pickle', './data/comments/20200201_20200528_all_comments_with_labels.pickle', './data/comments/20200315_20200528_all_comments_with_labels_dup.pickle', './data/comments/20200528_20200713_all_comments_with_labels.pickle', './data/comments/20200713_20200813_all_comments_roberta.pickle', './data/comments/20200713_20200813_all_comments_with_labels.pickle', './data/comments/20200813_20201025_all_comments_with_labels.pickle', './data/comments/20201025_20201128_all_comments_with_labels.pickle', './data/comments/20201129_20210201_all_comments_with_labels.pickle']


In [4]:
fname = files[3]
date = '20200528_20200713'
print(fname)
df = pd.read_pickle(fname)
df

./data/comments/20200528_20200713_all_comments_with_labels.pickle


,created_utc,parent_id,link_id,id,author,body,subreddit,link,score,author_flair_text,...,social,i,negate,you,posemo,negemo,anger,sad,we,swear
0,1594611009,t3_hq7drs,t3_hq7drs,fxw8pc6,Koalemos78,Interesting take from someone who's education ...,uspolitics,/r/uspolitics/comments/hq7drs/nm_state_senator...,3,NaN,...,7,1,1,0,0,0,0,0,0,0
1,1594610955,t1_fxl5isk,t3_hovua5,fxw8mi5,Thatsprettydank,Kinda lol they stopped researching out of bigotry,uspolitics,/r/uspolitics/comments/hovua5/well_be_living_w...,1,NaN,...,1,0,0,0,0,0,0,0,0,0
3,1594608113,t1_fxw2l39,t3_hq16qw,fxw4axz,Peripatetic_deviant,So you’re just guessing information? Even if t...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,...,2,0,0,1,0,0,0,0,0,0
4,1594607019,t1_fxvrvdq,t3_hq16qw,fxw2l39,buffalo_chum,They were out past curfew no? I bet there was ...,uspolitics,/r/uspolitics/comments/hq16qw/feds_in_portland...,1,NaN,...,0,0,1,0,0,0,0,0,0,0
5,1594606124,t3_hq4pq6,t3_hq4pq6,fxw153j,metaltimmy,Trump always likes these photo ops where he th...,uspolitics,/r/uspolitics/comments/hq4pq6/trump_ripped_as_...,5,NaN,...,16,0,1,2,10,7,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497708,1590724844,t1_fs3x7hl,t3_gs51kx,fs5yey2,matts2,[Conservapedia](https://www.conservapedia.com/...,Libertarian,/r/Libertarian/comments/gs51kx/psa_government_...,1,Mixed systems,...,0,0,0,0,0,0,0,0,0,0
4497711,1590724817,t1_fs5tuvq,t3_gshko8,fs5yde6,carnagebestvillain,Ah makes sense. Youre an authoritarian.\n\nLin...,Libertarian,/r/Libertarian/comments/gshko8/why_the_hell_ar...,1,NaN,...,2,0,0,2,0,1,1,0,0,0
4497712,1590724816,t3_gshb29,t3_gshb29,fs5ydbk,Effotless,The goal of the government is to protect our r...,Libertarian,/r/Libertarian/comments/gshb29/im_a_libertaria...,1,Anarcho-Objectivist,...,4,0,1,0,0,0,0,0,2,0
4497713,1590724814,t1_fs5vxr0,t3_gsieg3,fs5yd7o,Julian_Caesar,...well said.,Libertarian,/r/Libertarian/comments/gsieg3/the_minneapolis...,1,NaN,...,1,0,0,0,0,0,0,0,0,0


In [6]:
df.columns

Index(['created_utc', 'parent_id', 'link_id', 'id', 'author', 'body',
       'subreddit', 'link', 'score', 'author_flair_text', 'vulgarity',
       'personal_attacks', 'incivility', 'third_party_attacks', 'aspersion',
       'stereotype', 'comment_len', 'compound_sentiment', 'categories',
       'social', 'i', 'negate', 'you', 'posemo', 'negemo', 'anger', 'sad',
       'we', 'swear'],
      dtype='object')

In [10]:
categories = ['aspersion', 'civility', 'vulgarity', 'personal_attack', 'third_party_attack', 'stereotype']
files = ['', '_first', '_second', '_third']
for c in categories:
    print(c)
    labels = []
    for f in files:
        path = f'data/datasets/20200713_20200813_all_comments_with_labels{f}_roberta/{c}.npy'
        if os.path.isfile(path):
            print("there exists dataset under path " + path)
            with open(path, 'rb') as f:
                labels_dict = np.load(f, allow_pickle=True)
                acts = (F.softmax(torch.tensor(np.stack(labels_dict)))).cpu().detach().numpy()[:,1]
                labels.extend(acts)
    else:
      print("no dataset under path " + path)
    df[c] = labels
df

aspersion
there exists dataset under path data/datasets/20200713_20200813_all_comments_with_labels_roberta/aspersion.npy


/home/yujia/miniconda3/envs/incivility/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == "":


no dataset under path data/datasets/20200713_20200813_all_comments_with_labels_third_roberta/aspersion.npy


ValueError: Length of values (2834240) does not match length of index (4051806)